In [14]:
import matplotlib.pyplot as plt
import numpy as np
import imageio
import os



In [15]:
# Sphere
def Sphere_f(x):
    f = 0
    for i in x:
        f += i**2
    return f

# Michalewicz
def Michalewicz_f(x,m=10):
    d = len(x)
    j = np.arange(1, d+1)
    return -np.sum(np.sin(x) * np.sin(j*x**2/np.pi)**(2*m))

# Rosenbrock
def Rosenbrock_f(x):
    f = 0
    for i in range(len(x) - 1):
        f += (100*((x[i + 1] - x[i]**2)**2) + (1 -x[i])**2)
    return f

# Griewank
def Griewank_f(x):
    f1 = 0
    for i in x:
        f1 += (i*i)/4000
    f2 = 1
    for i in range(len(x)):
        f2 *= np.cos(x[i]/math.sqrt(i+1))
    return f1 - f2 + 1

# Ackley
def Ackley_f(x):
    a = 20
    b = 0.2
    c = 2 * np.pi
    f1 = 0
    f2 = 0
    for i in range(len(x)):
        f1 += x[i] ** 2
        f2 += np.cos(c * x[i])
    p1 = -a * np.exp(-b * np.sqrt(f1 / len(x)))
    p2 = -np.exp(f2 / len(x))

    return p1 + p2 + a + np.exp(1)

In [24]:
def DE(fobj, dimension, bounds, F_scale, cross_prob, popsize, max_evals):

    lower_bound, upper_bound = np.asarray(bounds).T

    eps = 0.00001

    diff = np.fabs(lower_bound - upper_bound)

    pop = lower_bound + diff * np.random.rand(popsize, dimension)

    fitness = np.asarray([fobj(ind) for ind in pop])
    num_eval = 1

    best_idx = np.argmin(fitness)
    best = pop[best_idx]

    results = []
    all_pops = []
    results.append((np.copy(best), fitness[best_idx], num_eval))
    all_pops.append(np.copy(pop))
    generation_count = 0
    while True:
        if num_eval > max_evals:
            break
        for i in range(popsize):
            fig, ax = plt.subplots(figsize=(8, 8))
            ax.axis('equal')
            # ax.contourf(pop, fitness, 90, cmap="viridis", alpha=0.7)
            ax.scatter(pop[:,0], pop[:,1], c="magenta")
            ax.scatter(pop[i][0], pop[i][1], c="red")
            plt.title('Generation ' + str(generation_count))
            idxes = [idx for idx in range(popsize) if idx != i]
            a, b, c = pop[np.random.choice(idxes, 3, replace=False)]
            ax.scatter(a[0], a[1], c="blue")
            ax.scatter(b[0], b[1], c="blue")
            ax.scatter(c[0], c[1], c="blue")
            mutant = np.clip(F_scale*(b - c) + a, lower_bound, upper_bound)
            ax.scatter(mutant[0], mutant[1], c="green")
            cross_points = np.random.rand(dimension) < cross_prob
            if not np.any(cross_points):
                cross_points[np.random.randint(0, dimension)] = True

            trial = np.where(cross_points, mutant, pop[i])
            ax.scatter(trial[0], trial[1], c="yellow")
            ax.plot(*minimum, "rD")
            plt.title('Generation ' + str(generation_count))
             # Lưu hình ảnh
            plt.savefig('images/gen_' + str(generation_count) + '.png')
            plt.close()
            f = fobj(trial)
            num_eval += 1

            if f < fitness[i]:
                pop[i] = trial
                fitness[i] = f
                if f < fitness[best_idx]:
                    best = trial
                    best_idx = i

        results.append((np.copy(best), fitness[best_idx], num_eval))
        all_pops.append(np.copy(pop))

        if fobj(best) < eps:
            num_eval += 1
            break

        generation_count += 1

        fig, ax = plt.subplots(figsize=(8, 8))
        ax.axis('equal')
      # ax.contourf(pop, fitness, 90, cmap="viridis", alpha=0.7)
        ax.scatter(pop[:,0], pop[:,1], c="magenta")
        ax.plot(*minimum, "rD")
        plt.title('Generation ' + str(generation_count))

        # Lưu hình ảnh
        plt.savefig('images/gen_' + str(generation_count) + '.png')
        plt.close()
    return results, all_pops, generation_count

In [9]:
F_scale=0.8 # hệ số scale F (0,1)
cross_prob=0.7 # xác suất hai cá thể đem đi lai ghép
# Tạo thư mục để lưu hình ảnh
if not os.path.exists('images'):
    os.makedirs('images')
all_results = []
d = 2
max_evals = 20000
f = Sphere_f
seed_number = 21522678
N = 32 # kích thước quần thể
lower_bound = -6
upper_bound = 6
np.random.rand(seed_number)
results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
with imageio.get_writer('Sphere_DE.gif', mode='I') as writer:
    for generation in range (0,generation_count):
        image = imageio.imread('/content/images/gen_' + str(generation) + '.png')
        writer.append_data(image)
for generation in range (0,generation_count):
    os.remove('/content/images/gen_' + str(generation) + '.png')

<ipython-input-9-42734af4e0a5>:18: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread('/content/images/gen_' + str(generation) + '.png')


In [ ]:
import math
F_scale=0.8 # hệ số scale F (0,1)
cross_prob=0.7 # xác suất hai cá thể đem đi lai ghép
# Tạo thư mục để lưu hình ảnh
if not os.path.exists('images'):
    os.makedirs('images')
all_results = []
d = 2
max_evals = 20000
f = Griewank_f
minimum = (0.0, 0.0)
seed_number = 21522678
N = 32 # kích thước quần thể
lower_bound = -6
upper_bound = 6
np.random.rand(seed_number)
results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
with imageio.get_writer('Griewank_DE.gif', mode='I') as writer:
    for generation in range (0,generation_count):
        image = imageio.imread('/content/images/gen_' + str(generation) + '.png')
        writer.append_data(image)
for generation in range (0,generation_count):
    os.remove('/content/images/gen_' + str(generation) + '.png')

<ipython-input-7-1f3bc845d719>:19: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread('/content/images/gen_' + str(generation) + '.png')


In [25]:
F_scale=0.8 # hệ số scale F (0,1)
cross_prob=0.7 # xác suất hai cá thể đem đi lai ghép
# Tạo thư mục để lưu hình ảnh
if not os.path.exists('images'):
    os.makedirs('images')
all_results = []
d = 2
max_evals = 20000
minimum = (1.0, 1.0)
f = Rosenbrock_f
seed_number = 21522678
N = 32 # kích thước quần thể
lower_bound = -6
upper_bound = 6
np.random.rand(seed_number)
results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
with imageio.get_writer('Rosenbrock_DE.gif', mode='I') as writer:
    for generation in range (0,generation_count):
        image = imageio.imread('/content/images/gen_' + str(generation) + '.png')
        writer.append_data(image)
for generation in range (0,generation_count):
    os.remove('/content/images/gen_' + str(generation) + '.png')

<ipython-input-25-b06dc4f7777e>:18: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread('/content/images/gen_' + str(generation) + '.png')


In [ ]:
F_scale=0.8 # hệ số scale F (0,1)
cross_prob=0.7 # xác suất hai cá thể đem đi lai ghép
# Tạo thư mục để lưu hình ảnh
if not os.path.exists('images'):
    os.makedirs('images')
all_results = []
d = 2
max_evals = 20000
f = Michalewicz_f
seed_number = 21522678
N = 32 # kích thước quần thể
lower_bound = -6
upper_bound = 6
np.random.rand(seed_number)
results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
with imageio.get_writer('Michalewicz_DE.gif', mode='I') as writer:
    for generation in range (0,generation_count):
        image = imageio.imread('/content/images/gen_' + str(generation) + '.png')
        writer.append_data(image)
for generation in range (0,generation_count):
    os.remove('/content/images/gen_' + str(generation) + '.png')

In [ ]:
F_scale=0.8 # hệ số scale F (0,1)
cross_prob=0.7 # xác suất hai cá thể đem đi lai ghép
# Tạo thư mục để lưu hình ảnh
if not os.path.exists('images'):
    os.makedirs('images')
all_results = []
d = 2
max_evals = 20000
minimum = (0.0, 0.0)
f = Ackley_f
seed_number = 21522678
N = 32 # kích thước quần thể
lower_bound = -6
upper_bound = 6
np.random.rand(seed_number)
results, all_pops, generation_count = DE(f, d, [(lower_bound, upper_bound)]*d, F_scale, cross_prob, N, max_evals)
with imageio.get_writer('Ackley_DE.gif', mode='I') as writer:
    for generation in range (0,generation_count):
        image = imageio.imread('/content/images/gen_' + str(generation) + '.png')
        writer.append_data(image)
for generation in range (0,generation_count):
    os.remove('/content/images/gen_' + str(generation) + '.png')

<ipython-input-5-c485a1af06f1>:18: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread('/content/images/gen_' + str(generation) + '.png')


In [21]:
import tensorflow as tf
def CEMv2(fobj, dimensions, bounds, popsize, num_elite, sigma_init, extra_std, seed_number):
    np.random.seed(seed_number)
    eps = 1e-4
    lower_bound, upper_bound = np.asarray(bounds).T
    sigma = sigma_init * np.eye(dimensions)

    diff = np.fabs(lower_bound - upper_bound)
    n_evals = 0
    num_evals = [0]
    mu = np.random.rand(dimensions) - (upper_bound + 1)
    generation_count = 0
    all_mu = []
    all_sigma = []
    all_offspring = []
    all_elite = []
    all_fitness = []
    while True:
    # for i in range(10000):
        if n_evals > max_evals:
            break
        all_mu.append(mu)
        all_sigma.append(sigma)

        x = np.random.multivariate_normal(mu, sigma, popsize)
        fitness = np.array([fobj(x[i]) for i in range(popsize)])
        all_offspring.append(x)
        x_lim = (0.5, 1.5)
        y_lim = (0.5, 1.5)
        n_seconds = 5
        n_points = 100
        xi = np.linspace(*x_lim, n_points)
        yi = np.linspace(*y_lim, n_points)
        X, Y = np.meshgrid(xi, yi)
        Z = fobj([X, Y])


        fig, ax = plt.subplots(figsize=(12, 12))
        ax.axis('equal')
        ax.contourf(X, Y, Z, 90, cmap="viridis", alpha=0.7)
        ax.scatter(x[:,0], x[:,1], c="magenta")



        n_evals += popsize
        best_fitness = min(fitness)
        all_fitness.append(best_fitness)
        # if np.sum(x) > 1e150 or np.sum(x) < -1e150:
        #     break
        if np.sum(x) > 1e150 or np.sum(x) < -1e150:
             break
        elite_idx = fitness.argsort()[:num_elite]
        selection = x[elite_idx]
        ax.scatter(selection[:,0], selection[:,1], c="cyan")
        plt.title('Generation ' + str(generation_count))
        ax.plot(*minimum, "rD")
        # Lưu hình ảnh
        plt.savefig('images/gen_' + str(generation_count) + '.png')
        plt.close()


        all_elite.append(elite_idx)
        sigma = np.zeros_like(sigma)

        for i in range(num_elite):
            z = x[elite_idx[i]] - mu
            z = z.reshape(-1, 1)
            sigma += tf.matmul(z.T, z)

        sigma += np.eye(dimensions)*extra_std
        sigma *= (1/num_elite)
        mu = np.mean(x[elite_idx], axis=0)

        generation_count += 1
        num_evals.append(n_evals)

    all_mu.append(mu)
    best_results = mu.copy()
    best_fitness = fobj(mu)
    return all_mu, all_fitness, num_evals, generation_count

In [12]:
d = 2
max_evals = 20000
f =   Sphere_f
seed_number = 21522679
N = 32 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01
minimum = (0.0, 0.0)
all_results = []
all_fitness = []
num_evals = []
results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
with imageio.get_writer('sphere_cem.gif', mode='I') as writer:
    for generation in range (0,generation_count):
        image = imageio.imread('/content/images/gen_' + str(generation) + '.png')
        writer.append_data(image)
for generation in range (0,generation_count):
    os.remove('/content/images/gen_' + str(generation) + '.png')

<ipython-input-12-d21eccc72e63>:18: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread('/content/images/gen_' + str(generation) + '.png')


In [17]:
if not os.path.exists('images'):
    os.makedirs('images')
import math
d = 2
max_evals = 20000
f =   Griewank_f
seed_number = 21522679
N = 32 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01
minimum = (0.0, 0.0)
all_results = []
all_fitness = []
num_evals = []
results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
with imageio.get_writer('Griewank_cem.gif', mode='I') as writer:
    for generation in range (0,generation_count):
        image = imageio.imread('/content/images/gen_' + str(generation) + '.png')
        writer.append_data(image)
for generation in range (0,generation_count):
    os.remove('/content/images/gen_' + str(generation) + '.png')

<ipython-input-17-78ff8231c62b>:21: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread('/content/images/gen_' + str(generation) + '.png')


In [22]:
d = 2
max_evals = 20000
f =   Rosenbrock_f
seed_number = 21522678
N = 32 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01
minimum = (1.0, 1.0)
all_results = []
all_fitness = []
num_evals = []
results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
with imageio.get_writer('Rosenbrock_cem.gif', mode='I') as writer:
    for generation in range (0,generation_count):
        image = imageio.imread('/content/images/gen_' + str(generation) + '.png')
        writer.append_data(image)
for generation in range (0,generation_count):
    os.remove('/content/images/gen_' + str(generation) + '.png')

<ipython-input-22-f4ff41e3637c>:18: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread('/content/images/gen_' + str(generation) + '.png')


In [ ]:
d = 2
max_evals = 20000
f =   Michalewicz_f
seed_number = 21522678
N = 32 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01
minimum = (0.0, 0.0)
all_results = []
all_fitness = []
num_evals = []
results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
with imageio.get_writer('Michalewicz_cem.gif', mode='I') as writer:
    for generation in range (0,generation_count):
        image = imageio.imread('/content/images/gen_' + str(generation) + '.png')
        writer.append_data(image)
for generation in range (0,generation_count):
    os.remove('/content/images/gen_' + str(generation) + '.png')

TypeError: unsupported operand type(s) for ** or pow(): 'list' and 'int'

In [ ]:
d = 2
max_evals = 20000
f =   Ackley_f
seed_number = 21522678
N = 32 # kích thước quần thể
lower_bound = -6
upper_bound = 6
num_elite = 10
sigma_init = 4
extra_std = 0.01
minimum = (0.0, 0.0)
all_results = []
all_fitness = []
num_evals = []
results, fitness, evals, generation_count = CEMv2(f, d, [(lower_bound, upper_bound)]*d, N, num_elite, sigma_init, extra_std, seed_number)
with imageio.get_writer('Ackley_cem.gif', mode='I') as writer:
    for generation in range (0,generation_count):
        image = imageio.imread('/content/images/gen_' + str(generation) + '.png')
        writer.append_data(image)
for generation in range (0,generation_count):
    os.remove('/content/images/gen_' + str(generation) + '.png')

<ipython-input-13-3044cf87e0c1>:18: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread('/content/images/gen_' + str(generation) + '.png')
